In [1]:
import gensim
# used for loading or saving
model_file = '/home/ramin/projects/ECO/src/python/modelbuilder/parsed_v3_valid.doc2vec'


done


In [105]:
# Build sentence list (each sentence needs at least 1 tag)
filename = '/home/marcel/drive/data/eco/NAIL_DATAFIELD_txt/parsed_v3/parsed_v3_valid.txt'

sentences = []
from random import shuffle

for uid, line in enumerate(open(filename)):
    ls = gensim.models.doc2vec.LabeledSentence(words=line.split(), tags=['SENT_%s' % uid])
    sentences.append(ls)
print(len(sentences),'sentences')

(4266193, 'sentences')


In [53]:
# Training the doc2vec model

# tutorial https://rare-technologies.com/doc2vec-tutorial/
# proposes shuffling or learning reate adjustment. we gonna do both
# in total 20 epochs
model = gensim.models.Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
print('building vocab') 
model.build_vocab(sentences)

base_alpha = model.alpha
base_min_alpha = model.min_alpha

for mepoch in range(2):
    model.alpha = base_alpha 
    model.min_alpha = base_min_alpha
    for epoch in range(10):
        print('epoch',mepoch * 10 + epoch)
        model.train(sentences)
        model.alpha -= 0.002  # decrease the learning rate
        model.min_alpha = model.alpha  # fix the learning rate, no decay
    shuffle(sentences)

# saving the model    
model.save(model_file)

building vocab
('epoch', 0)
('epoch', 1)
('epoch', 2)
('epoch', 3)
('epoch', 4)
('epoch', 5)
('epoch', 6)
('epoch', 7)
('epoch', 8)
('epoch', 9)
('epoch', 10)
('epoch', 11)
('epoch', 12)
('epoch', 13)
('epoch', 14)
('epoch', 15)
('epoch', 16)
('epoch', 17)
('epoch', 18)
('epoch', 19)


In [66]:
# Loading the model

model_loaded = gensim.models.Doc2Vec.load(model_file)

In [146]:
# Test: printing sentence 9 and getting the most similar ones.

print ' '.join(sentences[9][0])
sims = model_loaded.docvecs.most_similar('SENT_9')
print 'similar sentence',len(sims)
print '\nSIMILAR SENTENCES\n'
for sim in sims:
    print nice_print(sim)

Copyright Peter Abbs. Art Worlds in Schools by Professor Maxine Greene was first published in The Journal of Aesthetic Education , Vol .
similar sentence 10
 SIMILAR SENTENCES

Rather , art discloses the nature of things .
Heidi Rae Cooley is Assistant Professor of Media Arts in the Department of Art and Film and Media Studies Program at the University of South Carolina .
Rather , art discloses the nature of things .
You can see the final art in Figure 14.38 .
Art & Language and Victor Burgin exhibited at the Lisson Gallery .
See Art Buchwald et al. v. Paramount Pictures Corp .
New York : Museum of Modern Art Film Library .
Architecture is the first art of the frame .
See Art Buchwald et al. v. Paramount Pictures Corp .
Massachusetts Constitution of 1780 , Bill of Rights , arts .


In [143]:
# Tiny helper
def nice_print(tagged_doc):
    return ' '.join(sentences[int(tagged_doc[0][5:])][0])